Part 4, because now we're going to turn this into a full scale training loop, and i think getting to this step is super bad ass

In [9]:
import torch
import torch.nn.functional as F

In [10]:
words = open('names.txt', 'r').read().split()

In [17]:
N = torch.zeros((27, 27), dtype=torch.int32)
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(max(itos))

26


In [12]:
# Build your datasets
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
        
# then we'll turn these into tensors since we'll use them to build the network with PyTorch
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(f'number of examples: {xs.nelement()}')

number of examples: 228146


In [16]:
# Let's build our training loop

# initialize your random number generator from which you will obtain weights
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True) # remember to set requires_grad
print(max(xs))

train_steps = 10
for k in range(train_steps):
    # Forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()
    
    # Backward pass
    W.grad = None # remember to always zero your grads
    loss.backward()
    
    # Update data
    W.data += -50 * W.grad
    print(f'loss at step {k+1} of {train_steps}: {loss.item():.4}')

tensor(26)
loss at step 1 of 10: 3.759
loss at step 2 of 10: 3.371
loss at step 3 of 10: 3.154
loss at step 4 of 10: 3.02
loss at step 5 of 10: 2.928
loss at step 6 of 10: 2.86
loss at step 7 of 10: 2.81
loss at step 8 of 10: 2.77
loss at step 9 of 10: 2.738
loss at step 10 of 10: 2.711


We are supposed to be able to approach the original loss value that we got when we were just counting how many times a character would appear in the dataset afte smoothing, this was about 2.47, which is exactly what I'm getting now!

There's this awesome concept called regularization, which we can add onto our gradient descent algorithm to train our network. It's equivalent to making the smoothing in the `counting' scenario go from N+1 -> N+1e5, since all  this does is make all of the probabilities go to the uniform distribution

What we are after now is to be able to try to push all of the weights of W to go to zero. What this does is it sends the logits ==> exp.() to 1, so that the probabilities end up becoming the uniform distribution, and thus completely matching the uniform case for the counting scenario

In [14]:
# Let's build our training loop

# initialize your random number generator from which you will obtain weights
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True) # remember to set requires_grad

train_steps = 10
for k in range(train_steps):
    # Forward pass
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean() + 0.01*(W**2).mean()
    
    # Backward pass
    W.grad = None # remember to always zero your grads
    loss.backward()
    
    # Update data
    W.data += -50 * W.grad
    print(f'loss at step {k+1} of {train_steps}: {loss.item():.4}')

loss at step 1 of 10: 3.769
loss at step 2 of 10: 3.379
loss at step 3 of 10: 3.161
loss at step 4 of 10: 3.027
loss at step 5 of 10: 2.934
loss at step 6 of 10: 2.867
loss at step 7 of 10: 2.817
loss at step 8 of 10: 2.777
loss at step 9 of 10: 2.745
loss at step 10 of 10: 2.719


And now, let's sample from the neural network and see what kind of results we get!

In [47]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)
        
        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

junide.
janasah.
p.
cfay.
a.
